In [ ]:

from inspect import currentframe, getframeinfo


import pdb
#print frameinfo.filename, frameinfo.lineno


import traceback
import threading

import Chatbotaiml as ck
ss = input("Train Bot ?(Y/N)")
if "y" in ss or "Y" in ss:
    ck.trainbot()
# Python 3 support
try:
    from Queue import Queue
except ImportError:
    from queue import Queue

#print(getframeinfo(currentframe()).lineno) 

MULTIPROCESSING = 0.,
THREADS = 1

def tk_call_async(window, computation, args=(), kwargs={}, callback=None, errback=None, polling=500, method=MULTIPROCESSING):
    if method == MULTIPROCESSING:
        # I use threads because on windows creating a new python process freezes a little the event loop.
        future_result= Queue()

        worker = threading.Thread(target=_request_result_using_multiprocessing, args=(computation, args, kwargs, future_result))
        worker.daemon = True
        worker.start()
    elif method == THREADS:
        future_result = _request_result_using_threads(computation, args=args, kwargs=kwargs)
    else:
        raise ValueError("Not valid method")

    
    if callback is not None or errback is not None:
        _after_completion(window, future_result, callback, errback, polling)
        

    return future_result

def _request_result_using_multiprocessing(func, args, kwargs, future_result):
    import multiprocessing

    queue= multiprocessing.Queue()

    worker = multiprocessing.Process(target=_compute_result, args=(func, args, kwargs, queue))
    worker.daemon = True
    worker.start()

    return future_result.put(queue.get())

def _request_result_using_threads(func, args, kwargs):
    future_result= Queue()

    worker = threading.Thread(target=_compute_result, args=(func, args, kwargs, future_result))
    worker.daemon = True
    worker.start()

    return future_result


def _after_completion(window, future_result, callback, errback, polling):
    def check():
        try:
            result = future_result.get(block=False)
        except:
            window.after(polling, check)
        else:
            if isinstance(result, Exception):
                if errback is not None:
                    errback(result)
            else:
                if callback is not None:
                    if callback == "callback2":
                        callback2(result)
                    else:
                        callback(result)
                
    window.after(0, check)

def _compute_result(func, func_args, func_kwargs, future_result):
    try: 
        _result = func(*func_args, **func_kwargs)
        #_result = 
    except Exception as errmsg:
        _result = Exception(traceback.format_exc())

    future_result.put(_result)


# Multiprocessing uses pickle on windows.
# A pickable function should be in top module or imported from another module.
# This is requirement is not mandatory on Linux because python uses behind the scenes the fork operating system call.
# But on Windows it uses named pipes and pickle.

#import pyttsx3
'''
from  AppKit import NSSpeechSynthesizer
    

def speake(args):
    if not mixer.music.get_busy():
        print("inside speake")
        nssp = NSSpeechSynthesizer

        ve = nssp.alloc().init()

        ve.setVoice_("com.apple.speech.synthesis.voice.samantha")
        ve.startSpeakingString_(args)
    else:
        print("mixer is busy")
        
        
    engine = pyttsx3.init()
    volume =engine.getProperty('volume')
    engine.setProperty('volume',volume)
    engine.say(args)
    engine.runAndWait()
    engine.setProperty('rate',228)
    '''

def _example_calculation(args):
    #print("ex 0")
    #print(args,"Multithreading")
    #quess = jv.reset_dataset('ques')
    #ok = jv.changemusic(args,quess)
    ok = jv.checkinput(args)
    #ok = jv.test()
    #print(ok,"")
    if not ok:
        a =0
        for i in range(0,20000000):
            a+=1
        arg = ck.input_response("text",args)
        # user,bot
        ghb.add_chat(args,arg)
        return arg
    else:
        #print(args)
        arg = ok[0][2]
        ghb.add_chat(args,arg)
        #print(args)
        return arg

def _example1_calculation(args):
    #print("ex 1",args)
    ok = ""
    #ck.speake(args)
    
    return "ok"
    
    

if __name__ == "__main__":
    try:
        from tkinter import Tk, Frame, Entry, Label, Button, IntVar, StringVar, LEFT , Listbox , Scrollbar ,END , Checkbutton, filedialog
        import tkMessageBox as messagebox
    except ImportError:
        from tkinter import Tk, Frame, Entry, Label, Button, IntVar, StringVar, LEFT , Listbox , Scrollbar ,END , Checkbutton, filedialog
        from tkinter import messagebox
    
    from pygame import mixer
    import lsasimilarity as jv
    import re
    import TkTreectrl as treectrl
    import loudspeaker as ls
    import gethbasedata as ghb
    import quesanswer as qs
    global conversation
    conversation = False
    #ck.trainbot()
    #ok = jv.checkinput("What is active eco ?")
    mixer.init()
    
    disabledspk = False
    disabled = False
    
    def storebrwsdata(addres):
        quesans = []
        qs.addrs = addres
        qs.token = qs.sent_token(qs.addrs)
        ques,ans = qs.main()
        messagebox.showinfo("Information"," You are going to add "+ str(len(ques)) + " question and " + str(len(ans)) + " answer to the current dataset")
        for i in range (0,len(ques)):
            q = ques[i]
            a = ans[i]
            quesans.append(q+"\n"+a+"\n\n")
            print(q,"\n",a,"\n\n")
            # it is used to insert the question to the database table faq in HBase
            ghb.givedata(q , a)
        return quesans
        
    def trainstart():
        ques = lb.get(lb.curselection())
        ls = seperator(ques)
        ques = ls[1]
        if traincontent.get() == "":
            ans = llb.get(llb.curselection())
            print("Inside the trainconetent empty case")
        else:
            ans = traincontent.get()
            print("Inside the traincontetn no empty case")
        jv.insert2db(ques,ans)
        print(ques, "=>" , ans)
        messagebox.showinfo("Trained Sucessfully","Question :\n"+ ques + "\nAnswer:\n   " + ans)
        #lb.delete('0','end')
        llb.delete('0','end')
        llb2.delete('0','end')
        traincontent.set("")
        Curselet("edgd")
        print("Trained Sucessfully")
        
    
    
    def recostart():
        #print("Record started")
        #lblvar.set("Please speak")
        msgs = ck.listen()
        if not msgs == "I cannot hear you sir":
            msg = "You : " + msgs
            lb.insert(END,msg)
            calculate_fibonacci(msgs)
            sendtone()
            lb.yview(END)
        else:
            #print(msgs)
            a =0
            
            
    def calculate_fibonacci(msg):
        #print("fib 0")
        global disabled
        if disabled:
            messagebox.showinfo("warning", "Please wait Bot is thinking...")
            return

        def callback(result):
            #print("call 0")
            global disabled
            disabled = False
            insertb(result)
            #result_var.set(result)

        disabled = True
        
        tk_call_async(root, _example_calculation, args=(msg,), callback=callback, method =MULTIPROCESSING)
        
    '''def botspeak(msg):
        print("bot speak 1")
        global disabledspk
        if disabledspk:
            messagebox.showinfo("warning", "Please wait I am Speaking...")
            return

        def callback2(result):
            print("call 1")
            global disabledspk
            disabledspk = False
            
        disabledspk = True
        
        tk_call_async(root, speake, args=(msg,), callback=callback2, method =MULTIPROCESSING)
    
    '''

    
    root = Tk()
    
    def sendtone(): 
        mixer.music.load('/Users/prakash/send.mp3')
        mixer.music.play()
        

    def receivetone():
        mixer.music.load('/Users/prakash/receive.mp3')
        mixer.music.play()
    
    def gogo():
        result_var.set("Working")
        
    def insertb(msg):
        #msg = content.get()
        #print(ckbtn.get())
        #content.set("")
        msgs = "Bot : "+ msg 
        lb.insert(END,msgs)
        lb.yview(END)
        #ghb.add_chat('bot',msg)
        if ckbtn.get() == 1:
            ls.speakloud(msg)
            #receivetone()
        else:    
            receivetone()
        
    
    def seperator(string):
        aa = re.split(r' : ', string)
        if "You" in aa[0]:
            return True , aa[1]
        else:
            return False , aa[1]
        
    def insert():
        msgs = content.get()
        content.set("")
        msg = "You : " + msgs
        lb.insert(END,msg)
        calculate_fibonacci(msgs)
        #ghb.add_chat('user',msgs)
        sendtone()
        lb.yview(END)
    
    def insert2(args):
        check = []
        #ok = jv.checkinput(args)
        #print(ok,"Click Event")
        # ok is the list of [index, question, answer ,similarity]
        if ckbtn2.get() == 1:
            print("Checked LSA")
            #jv.lsachecker = True
            ok = jv.checkinputlsa(args)
        else:
            print("Not Checked LSA")
            #jv.lsachecker = False
            ok = jv.checkinput(args)
        if ok:
            mg = ok[0][2]
            for i in range (0,len(ok)):
                
                mm = ok[i][2]
                scr = flt_conv(ok[i][3])
                if mm not in check:
                    llb.insert(END,mm)
                    llb2.insert(END,scr)
                    check.append(mm)
        
    def Enter_event(event):
        '''
        if ckbtn2.get() == 1:
            print("Checked LSA")
            jv.lsachecker = True
        else:
            print("Not Checked LSA")
            jv.lsachecker = False
        '''
        msgs = content.get()
        content.set("")
        msg = "You : " + msgs
        lb.insert(END,msg)
        calculate_fibonacci(msgs)
        
        #ghb.add_chat('user',msgs)
        sendtone()
        lb.yview(END)
    
    def Enter_event_train(event):
        trainstart()
        #msgs = content.get()
        lb.yview(END)
    
    def flt_conv(flt):
        flt = flt *100
        jj = round(flt, 1)
        return str(jj) +"%"
    def browse_file():

        #if lblvar.get() =
        fname = filedialog.askopenfilename(filetypes = (("Template files", "*.txt"), ("All files", "*")))
        print (fname)
        qan = storebrwsdata(fname)
        joined = ' '.join(qan)
        messagebox.showinfo("Data set updated sucessfully", joined)
    
    def show_conversation():
        user , bot = ghb.getdataC()
        lb.delete(0,END)
        for i in range (0,len(user)):
            msgs = "You : "+ user[i] 
            lb.insert(END,msgs)
            msg = "Bot : "+ bot[i]
            lb.insert(END,msg)
        #messagebox.showinfo("conversation", "Sucessfully retrieved")
    
    def get_conversation():
        global conversation
        if conversation:
            lb.delete(0,END)
            conversation = False
        else:
            show_conversation()
            conversation = True
            
    root.title("Dialog Log")
    root.geometry("1100x630") # for outer window
    
    # for Right side  GUI
    
    n = IntVar(value=32)
    content = StringVar()    
    ckbtn = IntVar()
    ckbtn2 = IntVar()
    
    # main frame
    row = Frame(root, width=630, height=630, background="#EBEDEF") # for making frame
    
    scrollbar = Scrollbar(row)
    
    lb = Listbox(row, height=34, width=71, yscrollcommand=scrollbar.set)
    scrollbar.pack(side="right", fill="y")
    lb.yview(END) # viewing end message at first
    lb.pack(fill = "x",expand =1)  #packing listbox and filling to the frame x axis
    
    
    
    # first frame inside main frame
    rowin = Frame(row, width=630, height=20, background="#EBED22") # for making frame

    
    entry_field = Entry(rowin, width =68,textvariable =content)
    entry_field.pack(side=LEFT,fill = "x")
    entry_field.focus()
    entry_field.bind("<Return>", Enter_event)
    
    # Enter the query button
    Button(rowin, text="  Send  ", command = insert).pack(side= "right",fill = "both")
    
    rowin.pack(fill = "x")
    
    
    rowin2 = Frame(row, width=630, height=20) # for making frame
    
    abc = Button(rowin2,text="Reco",command= recostart).pack(side = "left")

    # Audio Out 
    Checkbutton(rowin2, text="Audio Out", variable=ckbtn).pack(side="right")
    
    # LSA Tick
    Checkbutton(rowin2, text="LSA", variable=ckbtn2).pack(side="right")
    
    # for browsing option
    broButton = Button(master = rowin2, text = 'Browse', width = 6, command=browse_file)
    broButton.pack(side="right", padx = 2, pady=2)
    
    broButton = Button(master = rowin2, text = 'Conversation', command=get_conversation)
    broButton.pack(side="right", padx = 2, pady=2)
    
    #lblvar = StringVar()
    #label = Label(rowin2, textvariable=lblvar, relief="raised")

    
    #label.pack(side="left")
    
    rowin2.pack(fill = "x")
    
    row.pack(side = "right")  #packing frame called row 
    
    def Curselet2(event):
        val = llb.get(llb.curselection())
        
        messagebox.showinfo("Message",val)
    
    def Curselet(event):
        val = lb.get(lb.curselection())
        #print(val)
        sts,mg = seperator(val)
        llb2.delete(0,END)
        llb.delete(0,END)
        if sts:
            insert2(mg)
        else:
            messagebox.showinfo("Message",mg)
    
    
    lb.bind('<<ListboxSelect>>',Curselet)
    
    traincontent = StringVar()
    
    
    row2 = Frame(root, width=470, height=580, background="#EBEDE3",borderwidth=1, relief="raised")
    
    row2l = Frame(row2)
    row2l.pack(side="left", expand="yes", fill="both")
    scrollbar2 = Scrollbar(row2)
    Label(row2l, text="Response",borderwidth=1, height =1 , relief="raised").pack(fill="x")
    llb = Listbox(row2l, width=38,height = 33, borderwidth=0, selectborderwidth=0,
         relief="flat", exportselection=False, yscrollcommand=scrollbar2.set)
    llb.pack(expand="yes", fill="both")
    row2r = Frame(row2)
    row2r.pack(side="left", expand="yes", fill="both")
    Label(row2r, text="Score",borderwidth=1,height = 1, relief="raised").pack(fill="x")
    llb2 = Listbox(row2r, width=9,height =33, borderwidth=0, selectborderwidth=0,
         relief="flat", exportselection=False, yscrollcommand=scrollbar2.set)
    llb2.pack(expand="yes", fill="both")
    llb2.configure(justify="right")
    
    scrollbar2.pack(side="right", fill = "y" )# packing scrollbar
    llb.bind('<<ListboxSelect>>',Curselet2)
    
    row2.pack()
    
    
    row3 =  Frame(root)
    
    #Training Button 
    trainb = Button(row3,text="Train",command= trainstart).pack(side = "left")
    
    #Entry field for Training 
    entry_field2 = Entry(row3, width =68,textvariable =traincontent)
    entry_field2.pack(side=LEFT,fill = "x")
    
    entry_field2.bind("<Return>", Enter_event_train)
    
    row3.pack()
    

    root.mainloop()

    #pdb.set_trace()
    '''
    scrollbar2 = Scrollbar(row2)# To navigate through past messages.
    
    lb2 = Listbox(row2,width = 44,height = 34, yscrollcommand=scrollbar2.set)
    
    scrollbar2.pack(side="right", fill = "y" )# packing scrollbar
    lb2.yview(END)
    lb2.pack()
    
    trainb = Button(row2,text="Train",command= trainstart).pack(side = "left")
    
    entry_field2 = Entry(row2, width =68,textvariable =traincontent)
    entry_field2.bind("<Return>", Enter_event_train)
    '''

Train Bot ?(Y/N)y
Loading from brain file: brain.dump
Loading brain from brain.dump...done (94243 categories in 2.53 seconds)
/Users/prakash/Google Drive/live/Study/WRC/MSCK III sem/Projects/Project Files/FAQ.txt
closed
closed
closed
closed
closed
closed
219
closed
closed
closed
closed
closed
220
closed
closed
closed
closed
closed
221
closed
closed
closed
closed
closed
closed
closed
closed
222
closed
closed
closed
closed
closed
closed
closed
closed
223
closed
closed
closed
closed
closed
closed
closed
closed
closed
closed
closed
closed
closed
closed
224
closed
closed
closed
closed
closed
closed
closed
closed
225
closed
closed
closed
closed
closed
closed
closed
closed
226
closed
closed
closed
closed
closed
227
closed
closed
closed
closed
closed
228
closed
closed
closed
closed
closed
closed
closed
closed
closed
229
Not Checked LSA
closed
closed
closed
closed
closed
closed
closed
closed
closed
closed
closed
closed
230
closed
closed
closed
closed
closed
231
closed
closed
closed
closed
close